created: Mon Jan  4 12:59:42 2021

**Group 5 - Classification**

**Features created for innovation detection**

@authors : Lilian Dulinge

In [1]:
import pandas as pd 
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
bow : pd.DataFrame = pd.read_csv("/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/BOW.csv")

In [6]:
bow.head()

,000,01,02,03,04,06,07,08,09,10,100,1000,11,110,118,11h,12,120,13,14,15,150,1500,16,17,18,180,19,1997,1998,1999,1er,20,200,2000,2001,2002,2003,2004,2005,...,étendue,éthique,étions,étranger,étrangers,étrangères,étroite,étroitement,étude,études,étudiant,étudiants,étudier,été,évaluation,évaluer,évalué,éventuellement,éventuelles,évidemment,évidence,évident,éviter,évolue,évoluent,évoluer,évolutif,évolution,évolutions,évolué,évoque,évoquer,évoqué,événement,événements,êtes,être,île,œuvre,art_id
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4


In [7]:
bow.shape

(12261, 4820)

In [8]:
Data : pd.DataFrame = pd.read_json("/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/Data.json")

In [9]:
Data.head()

,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,0,,None,7 décembre 2018,fr,4ème Conférence Nationale de l’Emploi Territor...,http://fncdg.com/4eme-conference-nationale-de-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2018/12/ra...,None,None
1,1,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,None,None
2,2,Malgré la levée des mesures de confinement le ...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,None,None
3,3,,None,28 juin 2017,fr,Arrêté de création GIP,http://fncdg.com/arrete-de-creation-gip/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/fo...,None,None
4,4,,None,24 novembre 2017,fr,Article Acteurs Publics « Les centres de gesti...,http://fncdg.com/article-acteurs-publics-les-c...,FNCDG,xpath_source,http://fncdg.com/actualites/,None,None,None


In [10]:
Data.shape

(12261, 13)

In [11]:
TfIdf : pd.DataFrame = pd.read_csv("/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/TF-IDF.csv")

In [12]:
TfIdf.head()

,000,01,02,03,04,06,07,08,09,10,100,1000,11,110,118,11h,12,120,13,14,15,150,1500,16,17,18,180,19,1997,1998,1999,1er,20,200,2000,2001,2002,2003,2004,2005,...,étendue,éthique,étions,étranger,étrangers,étrangères,étroite,étroitement,étude,études,étudiant,étudiants,étudier,été,évaluation,évaluer,évalué,éventuellement,éventuelles,évidemment,évidence,évident,éviter,évolue,évoluent,évoluer,évolutif,évolution,évolutions,évolué,évoque,évoquer,évoqué,événement,événements,êtes,être,île,œuvre,art_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.081753,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [13]:
TfIdf.shape

(12261, 4820)

In [14]:
Data['art_tag'].value_counts()

Not Found                                                                                                   1055
Actualités                                                                                                   572
['Retail & Fashion']                                                                                         564
['Urbanisme']                                                                                                467
['Institutions', 'et', 'pouvoirs', 'locaux']                                                                 467
                                                                                                            ... 
['Emploi - Travail', 'Organisation du travail']                                                                1
['Vie économique', "Gestion d'entreprise", 'Education - Recherche', 'Recherche développement']                 1
['Economie', 'Emploi - Travail', 'Salaire', 'Société', 'Santé - Protection sociale', 'Sécurité s

In [15]:
Data['art_lang'].value_counts()

fr    12146
en       10
Name: art_lang, dtype: int64

In [33]:
print("the ratio of non-empty tag is : " + str(Data['art_tag'].count()/Data.shape[0]) + " %")

the ratio of non-empty tag is : 0.8665687953674251 %


In [17]:
df_lexique : pd.DataFrame = pd.read_csv("/content/drive/MyDrive/G5 Inter-Promo 2021/Ressources/Lexique_Innovation.txt", sep="  ",header=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [18]:
df_lexique.columns = ['mots_cle']

In [20]:
stop_words = set(stopwords.words('french')) 
def preprocessing(data : pd.DataFrame) -> list:
  """Documentation
    Parameters In:
        data : 1 column of a dataframe especially choose because we want a column who contain only text

    Parameters Out :
        liste_return: liste of all the text of the dataframe return with preprocessing applied
    """
  liste_return = []
  pattern = ['(', ')', ':', ';', ',', '&', '/', '"', "'",'\n','©','n°','-']
  for i in tqdm(range(len(data))):
    sentence = data[i]
    if not isinstance(sentence,str):
        sentence  = str(sentence)
    
    sentence = re.sub(r'\d',' ',sentence)
    
    sentence = sentence.lower()

    for d in pattern:
        sentence = sentence.replace(d," ")
    
    token_words=word_tokenize(sentence)
    filtre = [w for w in token_words if not w in stop_words]

    final_sentence=[]
    for word in filtre:
        final_sentence.append(word)
        final_sentence.append(" ")
    liste_return.append("".join(final_sentence))
  return(liste_return)

In [21]:
def count_key_words(data : pd.DataFrame, liste : list) -> list:
  """Documentation
    Parameters In :
        data : One column of a dataframe who contains text for count word present in list
        liste : A list of key word which we want to see if they are present in the texts

    Parameters Out :
        list : list of numbers who represents the number of word in liste present in each texts
    """
  res = []
  for i in tqdm(range(len(data))):
    phrase = data[i]
    if phrase is None:
      res.append(0)
    else :
      phrase = phrase.lower()
      phrase = phrase.split()
      t = 0
      for j in phrase:
        if (j in list(liste)):
          t = t + 1
      res.append(t)
  return res

In [24]:
def get_numbers_words(data : pd.DataFrame, column : str) -> list:
    """Documentation
    Parameters In :
        data : dataframe which many columns
        column : name of the column on which we want to apply the function

    Parameters Out :
        list : list of numbers who represents the number of word in each line of the dataframe
    """
    size = []
    for i in tqdm(range(len(data[column]))):
      comm = data[column][i]

      if not isinstance(comm, str):
        comm = str(comm)

      if (comm is None):
        size.append(0)

      else:
        final = comm.split()
        size.append(len(final))
        
    return size

In [27]:
def sentences(data : pd.DataFrame, col : str) -> list:
    """Documentation
    Parameters In :
        data : dataframe which many columns
        column : name of the column on which we want to apply the function

    Parameters Out :
        list : list of numbers who represents the number of sentences in each line of the dataframe
    """
    liste = []  
    for i in tqdm(range(len(data[col]))):
      sentences = data[col][i]

      if not isinstance(sentences, str):
        sentences = str(sentences)

      if (sentences is None):
        count_sentence.append(0)
      else:
        sentences = sentences.replace("..", ".")
        sentences = sentences.replace("...", ".")
        sentences = sentences.replace("!", ".")
        sentences = sentences.replace("!!", ".")
        sentences = sentences.replace("!!!", ".")
        sentences = sentences.replace("?", ".")
        sentences = sentences.replace("??", ".")
        sentences = sentences.replace("???", ".")
        sentences = sentences.replace("?!", ".")
        sentences = sentences.replace("!?", ".")
        liste.append(len(sent_tokenize(sentences)))
    
    return liste

In [29]:
def Word_Apparition(data, word:str, column:str) -> list:
    """Documentation
    Parameters In :
        data : dataframe which many columns
        column : name of the column on which we want to apply the function
        word : the word we want to see is numbers of appearances in each lines

    Parameters Out :
        list : list of numbers who represents the number of appearances of a specific word in each documents
    """
    liste = []
    for i in tqdm(range(len(data[column]))):
        comm = data[column][i]
        iter = 0

        if not isinstance(comm, str):
          comm = str(comm)

        if (comm is None):
            liste.append(0)
        else:
          for j in range(len(comm)):
            if comm[j] == word:
              iter += 1
          liste.append(iter)
    return liste

In [30]:
Data['text_clean'] = preprocessing(Data['art_content'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [ ]:
#df_lexique['mots_cle'] = preprocessing(df_lexique['mots_cle'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [34]:
Data["Nb_key_words"] = count_key_words(Data["art_content"],df_lexique["mots_cle"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [35]:
Data["Nb_key_words_title"] = count_key_words(Data["art_title"],df_lexique["mots_cle"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [36]:
Data["Nb_words"] = get_numbers_words(Data, 'art_content')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [37]:
Data["Nb_words_title"] = get_numbers_words(Data, 'art_title')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [43]:
Data["Nb_sentences"] = sentences(Data, 'art_content')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [39]:
Data["exclamation"] = Word_Apparition(Data,'!', 'art_content') 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [40]:
Data["interrogation"] = Word_Apparition(Data,'?', 'art_content') 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


In [44]:
Data['ratio_words'] = Data['Nb_key_words']/Data['Nb_words']
Data['ratio_words'] = Data['ratio_words'].fillna(0)

In [45]:
Data['ratio_sentences'] = Data['Nb_key_words']/Data['Nb_sentences']
Data['ratio_sentences'] = Data['ratio_sentences'].fillna(0)

In [46]:
Data['ratio_title'] = Data['Nb_key_words_title']/Data['Nb_words_title']
Data['ratio_title'] = Data['ratio_title'].fillna(0)

In [47]:
Data.head(50)

,art_id,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag,text_clean,Nb_key_words,Nb_key_words_title,Nb_words,Nb_words_title,Nb_phrases,exclamation,interrogation,ratio_words,ratio_sentences,Nb_sentences,ratio_title
0,0,,None,7 décembre 2018,fr,4ème Conférence Nationale de l’Emploi Territor...,http://fncdg.com/4eme-conference-nationale-de-...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2018/12/ra...,None,None,,0,0,0,8,0,0,0,0.000000,0.000000,0,0.000000
1,1,La FNCDG et l’ANDCDG ont publié en septembre l...,"<p style=""text-align: justify;"">La FNCDG et l’...",22 septembre 2020,fr,9ème édition du Panorama de l’emploi territorial,http://fncdg.com/9eme-edition-du-panorama-de-l...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/09/im...,None,None,fncdg ’ andcdg publié septembre ème édition pa...,0,0,40,7,3,0,0,0.000000,0.000000,3,0.000000
2,2,Malgré la levée des mesures de confinement le ...,"<p style=""text-align: justify;"">Malgré la levé...",17 mars 2020,fr,ACTUALITÉS FNCDG / COVID19,http://fncdg.com/actualites-covid19/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2020/03/co...,None,None,malgré levée mesures confinement mai plupart m...,2,0,157,4,4,0,0,0.012739,0.500000,4,0.000000
3,3,,None,28 juin 2017,fr,Arrêté de création GIP,http://fncdg.com/arrete-de-creation-gip/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/05/fo...,None,None,,0,0,0,4,0,0,0,0.000000,0.000000,0,0.000000
4,4,,None,24 novembre 2017,fr,Article Acteurs Publics « Les centres de gesti...,http://fncdg.com/article-acteurs-publics-les-c...,FNCDG,xpath_source,http://fncdg.com/actualites/,None,None,None,,0,0,0,16,0,0,0,0.000000,0.000000,0,0.000000
5,5,,None,19 mars 2018,fr,Article AEF « Nous comptons profiter de la réf...,http://fncdg.com/article-aef-nous-comptons-pro...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2018/01/ac...,None,None,,0,0,0,21,0,0,0,0.000000,0.000000,0,0.000000
6,6,,None,24 novembre 2017,fr,"Article la Gazette des Communes « Les CDG, une...",http://fncdg.com/article-la-gazette-des-commun...,FNCDG,xpath_source,http://fncdg.com/actualites/,None,None,None,,0,0,0,15,0,0,0,0.000000,0.000000,0,0.000000
7,7,,None,17 janvier 2018,fr,Article La Gazette des Communes « Les centres ...,http://fncdg.com/article-la-gazette-des-commun...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2018/01/ac...,None,None,,0,0,0,17,0,0,0,0.000000,0.000000,0,0.000000
8,8,,None,13 mars 2018,fr,Article L’Ingénierie Territoriale « Les CDG « ...,http://fncdg.com/article-lingenierie-territori...,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2018/03/in...,None,None,,0,0,0,19,0,0,0,0.000000,0.000000,0,0.000000
9,9,,None,10 octobre 2017,fr,Baromètre HoRHizons 2017,http://fncdg.com/barometre-horhizons-2017/,FNCDG,xpath_source,http://fncdg.com/actualites/,http://fncdg.com/wp-content/uploads/2017/10/Ho...,None,None,,0,0,0,3,0,0,0,0.000000,0.000000,0,0.000000
